# ResNet50 与 BERT 预训练模型的域内微调与保存

目的：先用电商数据中的所有图片和文本，分别对预训练好的 ResNet50 与 BERT 进行域内微调（自监督/MLM），然后将微调后的权重保存，供后续多模态检索任务使用。

- 图像侧：使用自监督旋转预测任务（0/90/180/270）对 ResNet50 的顶层进行轻量微调。
- 文本侧：使用 Masked Language Modeling (MLM) 在电商查询文本上微调 BERT。
- 结果：保存两个模型的微调后权重，后续检索任务可直接加载。


In [1]:
import os
import math
import json
from typing import List

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader as TorchDataLoader
from torch.optim import Adam, AdamW

import timm
from torchvision import transforms
from torchvision.transforms import functional as TF
from PIL import Image
from tqdm import tqdm
from torch.amp import autocast, GradScaler

from transformers import BertTokenizer, BertForMaskedLM, DataCollatorForLanguageModeling, get_cosine_schedule_with_warmup

# 环境与缓存设置（与项目保持一致，可按需修改为本地镜像/缓存）
cache_dir = "/mnt/d/HuggingFaceModels/"
os.environ['TORCH_HOME'] = cache_dir
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "1"
os.environ['https_proxy'] = 'http://127.0.0.1:7890'
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['all_proxy'] = 'socks5://127.0.0.1:7890'
os.environ["WANDB_DISABLED"] = "true"
os.environ['CURL_CA_BUNDLE'] = ""
os.environ['HF_ENDPOINT'] = "https://hf-mirror.com"

# 导入项目数据加载器
import sys
sys.path.append(os.path.abspath(os.path.join('.', 'Multimodal_Retrieval', 'plan1_1')))
from data_loader import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# 保存路径
save_dir = '/mnt/d/forCoding_data/Tianchi_MUGE/trained_models/weights'
os.makedirs(save_dir, exist_ok=True)
resnet_save_path = os.path.join(save_dir, 'ft_resnet50_rotation_backbone.pth')
bert_save_path = os.path.join(save_dir, 'ft_bert_base_chinese_mlm.pth')


/home/xiuminke/miniconda3/envs/ml12/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/home/xiuminke/miniconda3/envs/ml12/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [2]:
import time
while True:
    if os.path.exists("step_2_3-3_屯-mean_pooling-v2_cp1.finishflag"):
        break
    time.sleep(5)

In [3]:
# 初始化数据加载器
loader = DataLoader()
## batch_size=1000, image_size=(224, 224)

2025-11-09 21:30:40,249 - INFO - 初始化数据加载器，数据目录: /mnt/d/forCoding_data/Tianchi_MUGE/originalData/Multimodal_Retrieval


## 图像侧：ResNet50 旋转预测自监督微调
- 使用 timm 的 ResNet50 特征提取模式（`num_classes=0`），只微调 `layer4` 顶层。
- 在每张图像上构造 4 个旋转版本，对应标签 0/1/2/3。
- 在分类头上计算交叉熵，优化 `layer4` 与分类头。


In [4]:
from safetensors.torch import load_file

class RotationHead(nn.Module):
    def __init__(self, in_dim=2048, num_classes=4):
        super().__init__()
        self.fc = nn.Linear(in_dim, num_classes)
    def forward(self, x):
        return self.fc(x)

def build_resnet50_backbone(device, cache_dir=None, weights_path=None):
    model = timm.create_model('resnet50', pretrained=False, num_classes=0, cache_dir=cache_dir).to(device)
    if weights_path is not None:
        if weights_path.endswith('.safetensors'):
            state_dict = load_file(weights_path)
        else:
            state_dict = torch.load(weights_path, map_location='cpu')
        model.load_state_dict(state_dict, strict=False)

    
    # 仅微调 layer4 顶层参数
    for p in model.parameters():
        p.requires_grad = False
    if hasattr(model, 'layer4'):
        for p in model.layer4.parameters():
            p.requires_grad = True
    return model

class RotationBatchDataset(Dataset):
    def __init__(self, items, transform, angles=(0,90,180,270)):
        self.transform = transform
        self.angles = angles
        self.samples = []
        for it in items:
            img = it.get('image', None)
            if img is None:
                continue
            for li, ang in enumerate(self.angles):
                self.samples.append((img, li, ang))
    def __len__(self):
        return len(self.samples)
    def __getitem__(self, idx):
        img, li, ang = self.samples[idx]
        rotated = TF.rotate(img, angle=ang)
        return self.transform(rotated), li

def train_resnet_rotation(loader: DataLoader, device, epochs=1, img_batch_size=800, inner_bs=24, lr=1e-4, max_batches=2, use_amp=True, warmup_ratio=0.05):
    model = build_resnet50_backbone(device=device, cache_dir=cache_dir, 
                                   weights_path="/mnt/d/HuggingFaceModels/models--timm--resnet50.a1_in1k/snapshots/767268603ca0cb0bfe326fa87277f19c419566ef/model.safetensors"
                                   )
    head = RotationHead(in_dim=2048, num_classes=4).to(device)
    params = list(model.layer4.parameters()) + list(head.parameters()) if hasattr(model, 'layer4') else list(head.parameters())
    optim = Adam(params, lr=lr)
    # 预估调度步数：每张原图生成4个旋转样本，inner_bs为步长
    est_imgs = (img_batch_size if max_batches is None else img_batch_size * max_batches)
    total_train_samples = est_imgs * 4
    est_total_steps = max(1, total_train_samples // inner_bs)
    num_warmup_steps = max(1, int(est_total_steps * warmup_ratio))
    scheduler = get_cosine_schedule_with_warmup(optim, num_warmup_steps=num_warmup_steps, num_training_steps=est_total_steps)
    scaler = GradScaler(enabled=(use_amp and device.type=='cuda'))
    criterion = nn.CrossEntropyLoss()
    base_tf = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    angles = [0, 90, 180, 270]

    model.train()
    outer = loader.load_images_batch(split='train', batch_size=img_batch_size, max_batches=max_batches)
    for batch_idx, img_batch in enumerate(outer):
    # for batch_idx, img_batch in enumerate(tqdm(outer, desc="ResNet rotation: image batches", total=max_batches if max_batches else None)):
        ds = RotationBatchDataset(img_batch, base_tf, angles)
        if len(ds) == 0:
            continue
        dl = TorchDataLoader(ds, batch_size=inner_bs, shuffle=True, num_workers=2, pin_memory=(device.type=='cuda'))
        running_loss = 0.0
        steps = 0
        for xb, yb in dl: # tqdm(dl, desc=f"Batch {batch_idx+1}"):
            xb = xb.to(device, non_blocking=True)
            yb = torch.tensor(yb, dtype=torch.long, device=device)
            with autocast(
                device_type = device.type, 
                enabled=(use_amp and device.type=='cuda')
            ):
                feats = model(xb)
                logits = head(feats)
                loss = criterion(logits, yb)
            optim.zero_grad(set_to_none=True)
            if use_amp and device.type=='cuda':
                scaler.scale(loss).backward()
                torch.nn.utils.clip_grad_norm_(params, max_norm=1.0)
                scaler.step(optim)
                scaler.update()
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(params, max_norm=1.0)
                optim.step()
            scheduler.step()
            running_loss += float(loss.detach().item())
            steps += 1
        if steps > 0:
            current_lr = optim.param_groups[0]['lr']
            print(f"Rotation Batch {batch_idx+1}: avg loss={running_loss/steps:.4f}, lr={current_lr:.6f}")
        if device.type == 'cuda':
            torch.cuda.empty_cache()
    return model


In [5]:
# 1) 图像侧：ResNet50 旋转预测自监督微调
resnet_model = train_resnet_rotation(
    loader, device=device, img_batch_size=1000, max_batches=200, inner_bs=24, epochs=2, lr=1e-4, use_amp=True
)
torch.save(resnet_model.state_dict(), resnet_save_path)
print(f'ResNet50 backbone fine-tuned and saved to: {resnet_save_path}')

2025-11-09 21:30:40,779 - INFO - 批量加载train图片数据: /mnt/d/forCoding_data/Tianchi_MUGE/originalData/Multimodal_Retrieval/MR_train_imgs.tsv
实际加载train图片数据:   1%|▍                                                    | 938/129380 [00:00<00:40, 3152.04it/s]/tmp/ipykernel_207347/3530071927.py:80: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  yb = torch.tensor(yb, dtype=torch.long, device=device)
实际加载train图片数据:   1%|▌                                                    | 1254/129380 [00:03<08:45, 243.78it/s]

Rotation Batch 1: avg loss=1.3872, lr=0.000010


实际加载train图片数据:   2%|▊                                                    | 2017/129380 [00:06<09:57, 213.21it/s]

Rotation Batch 2: avg loss=1.3873, lr=0.000020


实际加载train图片数据:   3%|█▎                                                   | 3246/129380 [00:09<08:10, 257.00it/s]

...

Rotation Batch 128: avg loss=0.8587, lr=0.000031


实际加载train图片数据: 100%|██████████████████████████████████████████████████▉| 129086/129380 [06:15<00:01, 231.00it/s]

Rotation Batch 129: avg loss=0.8044, lr=0.000030


实际加载train图片数据: 100%|███████████████████████████████████████████████████| 129380/129380 [06:15<00:00, 344.38it/s]


Rotation Batch 130: avg loss=0.7684, lr=0.000030
ResNet50 backbone fine-tuned and saved to: /mnt/d/forCoding_data/Tianchi_MUGE/trained_models/weights/ft_resnet50_rotation_backbone.pth


## 文本侧：BERT 在电商查询上的 MLM 微调
- 收集 train/valid 查询文本，构造语料。
- 使用 `BertForMaskedLM` 与 `DataCollatorForLanguageModeling` 实现标准 MLM 训练。
- 默认微调整个 BERT；可选仅微调顶层。


In [6]:
class TextDataset(Dataset):
    def __init__(self, texts: List[str], tokenizer: BertTokenizer, max_length=128):
        self.texts = texts
        self.tok = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        enc = self.tok(text, return_tensors='pt', padding='max_length', truncation=True, max_length=self.max_length)
        # squeeze to remove batch dim
        item = {k: v.squeeze(0) for k, v in enc.items()}
        return item

def train_bert_mlm(loader: DataLoader, device, epochs=1, batch_size=64, lr=5e-5, max_samples=None, last_n_layers=None, use_amp=True, warmup_ratio=0.05):
        tokenizer = BertTokenizer.from_pretrained('bert-base-chinese', cache_dir=cache_dir, local_files_only=True)
        model = BertForMaskedLM.from_pretrained('bert-base-chinese', cache_dir=cache_dir, local_files_only=True).to(device)
        # 可选：仅微调顶层
        if last_n_layers is not None:
            for p in model.bert.parameters():
                p.requires_grad = False
            enc = model.bert.encoder
            total_layers = len(enc.layer)
            start_idx = max(0, total_layers - last_n_layers)
            for i in range(start_idx, total_layers):
                for p in enc.layer[i].parameters():
                    p.requires_grad = True
                enc.layer[i].train()
            if hasattr(model.bert, 'pooler') and model.bert.pooler is not None:
                for p in model.bert.pooler.parameters():
                    p.requires_grad = True
                model.bert.pooler.train()
        # 数据准备
        train_df = loader.load_queries(split='train')
        valid_df = loader.load_queries(split='valid')
        train_texts = [t for t in train_df.get('query_text', []) if isinstance(t, str)]
        valid_texts = [t for t in valid_df.get('query_text', []) if isinstance(t, str)]
        if max_samples is not None:
            train_texts = train_texts[:max_samples]
        ds_train = TextDataset(train_texts, tokenizer)
        collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
        dl_train = TorchDataLoader(ds_train, batch_size=batch_size, shuffle=True, collate_fn=collator, num_workers=2, pin_memory=(device.type=='cuda'))
        optim = AdamW([p for p in model.parameters() if p.requires_grad], lr=lr)
        # 预估总步数：按最大样本与批大小粗略估算
        est_train_samples = len(train_texts) if max_samples is None else min(len(train_texts), max_samples)
        est_total_steps = max(1, (est_train_samples // batch_size) * epochs)
        num_warmup_steps = max(1, int(est_total_steps * warmup_ratio))
        scheduler = get_cosine_schedule_with_warmup(optim, num_warmup_steps=num_warmup_steps, num_training_steps=est_total_steps)
        scaler = GradScaler(enabled=(device.type=='cuda'))
        model.train()
        for e in range(epochs):
            step = 0
            running_loss = 0.0
            for batch in tqdm(dl_train, desc=f"MLM epoch {e+1}/{epochs}"):
                batch = {k: v.to(device) for k, v in batch.items()}
                with autocast(
                    device_type = device.type, 
                    enabled=(use_amp and device.type=='cuda')
                ):
                    outputs = model(**batch)
                    loss = outputs.loss
                optim.zero_grad(set_to_none=True)
                scaler.scale(loss).backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                scaler.step(optim)
                scaler.update()
                scheduler.step()
                step += 1
                running_loss += float(loss.detach().item())
            if step > 0:
                current_lr = optim.param_groups[0]['lr']
                print(f"MLM epoch {e+1}: avg loss={running_loss/step:.4f}, lr={current_lr:.6f}")
            # 简单的验证（可选）：这里只做一次前向以确保正常
            if valid_texts:
                sample_texts = valid_texts[:8]
                ds_valid = TextDataset(sample_texts, tokenizer)
                dl_valid = TorchDataLoader(ds_valid, batch_size=8, shuffle=False, collate_fn=collator)
                model.eval()
                with torch.no_grad():
                    for vb in dl_valid:
                        _ = model(**{k: v.to(device) for k, v in vb.items()})
                model.train()
        return model


In [7]:
# 2) 文本侧：BERT MLM 微调
bert_model = train_bert_mlm(
    loader, device=device, epochs=2, batch_size=64, lr=5e-5, max_samples=300000, last_n_layers=None, use_amp = True
)
torch.save(bert_model.state_dict(), bert_save_path)
print(f'BERT MLM fine-tuned and saved to: {bert_save_path}')


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2025-11-09 21:37:08,554 - INFO - 加载train查询数据: /mnt/d/forCoding_data/Tianchi_MUGE/originalData/Multimodal_Retrieval/MR_train_queries.jsonl
加载train查询数据: 248786it [00:00, 269232.52it/s]
2025-11-09 21:37:09,540 - INFO - 成功加载train查询数据，共248786条
2025-11-09 21:37:09,552 - INFO - 加载valid查询数据: /mnt/d/forCoding_data/Tianch

MLM epoch 1: avg loss=2.9190, lr=0.000027


MLM epoch 2/2: 100%|████████████████████████████████████████████████████████████████| 3888/3888 [07:56<00:00,  8.15it/s]


MLM epoch 2: avg loss=2.5924, lr=0.000000
BERT MLM fine-tuned and saved to: /mnt/d/forCoding_data/Tianchi_MUGE/trained_models/weights/ft_bert_base_chinese_mlm.pth


## 后续使用建议
- ResNet50：后续检索任务中，使用 `timm.create_model('resnet50', num_classes=0)` 并加载上述 `state_dict`；只取特征作为图像嵌入。
- BERT：后续检索任务中，加载 `BertModel` 并将 `BERT MLM` 的 `state_dict` 映射到 `bert.*` 对应权重（本 notebook 保存的是 `BertForMaskedLM` 的整体权重）。
- 若需更长训练或更强微调：适当增大 `max_batches`（图像）与 `epochs`（文本），并根据显存与时间预算调整 `batch_size`。


In [8]:
with open("ft_rsn50_brt.finishflag", "w") as f:
    f.write("finish")

import IPython
def kill_current_kernel():
    '''杀死当前的kernel释放内存空间。'''
    IPython.Application.instance().kernel.do_shutdown(True)
kill_current_kernel()